# Prototype Development

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

from pathlib import Path

import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

from utils import *
from plots import *

data_path = Path("..") / "data"
counts_path = data_path / "Pedestrian_Counting_System___2009_to_Present__counts_per_hour_.csv"
sensor_path = data_path / "Pedestrian_Counting_System_-_Sensor_Locations.csv"

First we'll prep some dataset metadata that we'll need: 

In [3]:
foot_df = load_and_clean_pedestrian_data(counts_path, lat_long_path=sensor_path)

In [4]:
import calendar

MONTHS = list(calendar.month_name)[1:]
WEEKDAYS = list(calendar.day_name)
YEARS = sorted(foot_df["Year"].unique())
SENSORS = sorted(foot_df["Sensor_Name"].unique())
NUM_SNESORS = len(SENSORS)

## Ipywidgets

* input controls
* display outputs
* binding inputs and outputs to interactions

In [5]:
import ipywidgets as widgets

In [6]:
widgets.Dropdown(options=SENSORS)

Dropdown(options=('231 Bourke St', 'Alfred Place', 'Australia on Collins', 'Birrarung Marr', 'Bourke St - Spen…

In [7]:
widgets.Dropdown(options=YEARS)

Dropdown(options=(2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020), value=2009)

In [8]:
first_year = min(YEARS)
last_year = max(YEARS)
widgets.IntRangeSlider(
    value=[first_year, last_year],
    min=first_year,
    max=last_year,
)

IntRangeSlider(value=(2009, 2020), max=2020, min=2009)

## Prototype

In [147]:
# TODO
# -- diagnose and fix issues with souns count/ sensor time series plots when one/few plots displayed
# -- initial month/sensor values need constraining
# -- add titles with current filter values to each plot
# -- sort out margins
# -- address artefcts in plots
# -- add prev 4 week average to time series plots etc
# -- see if any options for caching

from ipywidgets import VBox, HBox, Dropdown, SelectMultiple, HTML, Output


def prototype(df, debug=False):
    """Create a prototype Dashboard"""
    # inputs
    year_input = Dropdown(options=YEARS, value=YEARS[-1], description="Year")
    month_input = Dropdown(options=MONTHS, value=None, description="Month")
    sensor_input = SelectMultiple(options=SENSORS, value=[], description="Sensor", layout={"height":"150px"})

    def update_inputs(change):
        """Update available options for month and sensor inputs based on current year"""
        filtered_df = filter_foot_df(df, year=year_input.value)
        # Need to temporarily disable handlers all other custom handlers
        # to prevent input value/option changes triggering a UI ipdate
        month_handlers = month_input._trait_notifiers["value"]["change"]
        month_input._trait_notifiers["value"]["change"] = []
        month_input.options = filtered_df["Month"].unique()
        month_input.value = None
        month_input._trait_notifiers["value"]["change"] = month_handlers

        sensor_handlers = sensor_input._trait_notifiers["value"]["change"]
        sensor_input._trait_notifiers["value"]["change"] = []
        sensor_input.options = sorted(filtered_df["Sensor_Name"].unique())
        sensor_input.value = []
        sensor_input._trait_notifiers["value"]["change"] = sensor_handlers

    if debug:
        debug_view = Output(layout={"border": "1px solid black"})
        update_inputs = debug_view.capture(clear_output=True)(update_inputs)

    year_input.observe(update_inputs, "value")

    # outputs
    month_counts_output = widgets.interactive_output(
        display_output(functools.partial(plot_month_counts, df, height=350, width=550)),
        {"sensor": sensor_input, "year": year_input},
    )
    sensor_counts_output = widgets.interactive_output(
        display_output(functools.partial(plot_sensor_counts, df, width=550)),
        {"sensor": sensor_input, "year": year_input, "month": month_input},
    )
    sensors_output = widgets.interactive_output(
        display_output(
            functools.partial(plot_sensors, df, same_yscale=True, width=1200)
        ),
        {"sensor": sensor_input, "year": year_input, "month": month_input},
    )
    sensors_map_output = widgets.interactive_output(
        display_output(functools.partial(plot_scatter_map, df, height=650, width=600)),
        {"sensor": sensor_input, "year": year_input, "month": month_input},
    )

    # layout
    title = HTML("<H1>Melbourne CBD Pedestrian Traffic</h1>")
    inputs = VBox([year_input, month_input, sensor_input])
    col1_row1 = HBox([VBox([HBox([inputs]), month_counts_output]), sensors_map_output])
    col1_row2 = sensors_output
    col1 = VBox([col1_row1, col1_row2])
    col2 = sensor_counts_output
    rows = [title, HBox([col1, col2])]

    if debug:
        debug_row = HBox([debug_view])
        rows.insert(0, debug_row)

    return VBox(rows)


prototype(foot_df, debug=True)

## Comparing Sensors by Year

In [ ]:
import functools

# inputs
sensor_input = widgets.Dropdown(options=SENSORS)
year_input = widgets.Dropdown(options=YEARS)
month_input = widgets.Dropdown(options=MONTHS)

# binding inputs to output function
out = widgets.interactive_output(
    display_output(functools.partial(plot_years, foot_df)),
    {"sensor": sensor_input, "month": month_input},
)

# layout
widgets.HBox(
    [
        widgets.VBox(
            [
                widgets.HTML("<H1>Comparing Sensors by Year</h1>"),
                sensor_input,
                month_input,
            ]
        ),
        out,
    ]
)

Ideas:
* align to eg first Monday of the month
* get rid of the 2000 from the initial x value see `tickformatstops` example near bottom of [this page](https://plotly.com/python/time-series/)
* extend to `sensor_plot`
* add ability to overlay mean/median lines from custom date (ranges)
* work out how to combine `plot_sensors` and `plot_year` into single `plot_comparison` that also allows selecting month, and even day?
* improve hover label

In [ ]:
is_str_or_non_empty([1,2,4])

In [ ]:
import numpy as np
import numbers

In [ ]:
isinstance(np.array([1])[0], numbers.Number)

In [ ]:
isinstance(np.array([1]), collections.abc.MutableSequence)